In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.ndimage import gaussian_filter1d
import json
from scipy.special import expit
from scipy import signal
from scipy.optimize import curve_fit
import re
from more_itertools import windowed
from scipy.signal import find_peaks, correlate

In [83]:
def load_json_data(path):
    # Opening JSON file
    f = open(path)

    # returns JSON object as a dictionary
    data = json.load(f)
    df = pd.json_normalize(data)
    
    ### ONLY FOR THIS EXPERIMEMT, DOUBLE CHECK EMPTY LISTS ###
    df = df.drop(1,axis=0).reset_index().drop(columns='index')

    final_list_of_fish_dicts = []
    for i, r in df.iterrows():
        fish_dict = {}
        print(i)
        for stim_type in df.columns:
            print(stim_type)
            fish_dict[stim_type] = []
            for counter in range(len(df[stim_type])-1):
                dfdf = df[stim_type][i][0]
                dfdfdf = json.loads(dfdf)
                norm_df = pd.json_normalize(dfdfdf, max_level=0)
                empty_np = np.zeros((200, ))
                for col in norm_df.columns:
                    col_array = np.array([pd.json_normalize(norm_df[col])][0])[0]
                    if col_array.shape == (199, ):
                        col_array = np.insert(col_array, 0, 0)
                    empty_np = np.vstack((empty_np, col_array))
                final_df = pd.DataFrame(empty_np)
                final_df = final_df.transpose().drop(columns=[0])
                final_df.columns = norm_df.columns
                fish_dict[stim_type].append(final_df)
        final_list_of_fish_dicts.append(fish_dict)

    return final_list_of_fish_dicts

In [84]:
list_of_fish_dicts = load_json_data('../json_data/STIM_2_3_4_AMPLROT_DIST.json')
len(list_of_fish_dicts)

0
STIM_type_0.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_0.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.05.AMPL_rot_1.047198
1
STIM_type_0.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_0.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.05.AMP

STIM_type_3.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.05.AMPL_rot_1.047198
12
STIM_type_0.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_0.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.05.AMPL_rot_1.047198
13
STIM_type_0.DIST_dot_0.1.

STIM_type_2.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.05.AMPL_rot_1.047198
24
STIM_type_0.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_0.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.05.AMPL

STIM_type_4.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.05.AMPL_rot_1.047198
35
STIM_type_0.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_0.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.05.AMPL_rot_1.047198
36
STIM_type_0.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_0.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.1.

STIM_type_3.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.05.AMPL_rot_1.047198
47
STIM_type_0.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_0.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_0.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_2.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_2.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_3.DIST_dot_0.05.AMPL_rot_0.5235989
STIM_type_3.DIST_dot_0.05.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.1.AMPL_rot_0.5235989
STIM_type_4.DIST_dot_0.1.AMPL_rot_1.047198
STIM_type_4.DIST_dot_0.05.AMPL

48

In [86]:
fish0 = list_of_fish_dicts[0]

In [98]:
fish0 = list_of_fish_dicts[0]
test_frame = fish0['STIM_type_0.DIST_dot_0.05.AMPL_rot_0.5235989'][2]
test_frame

,Timestamp,X,Y,ANGLE,CUM_angle,TAIL_P1,TAIL_P2,TAIL_P3,TAIL_P4,TAIL_P5,...,STIM_type,SIZE_dot,DIST_dot,AMPL_rot,SPEED_rot,LUM_dot,BGLUM,Counter,Distance_pts,Relative_stim_angle
0,0.02,295.330013,545.497877,6.040285,0.000000,0.0,-0.022852,0.017680,0.017681,0.017680,...,0.0,0.004,0.05,0.523599,0.35,1.0,0.0,14.0,0.000000,0.0
1,0.04,295.069727,545.429028,6.040393,0.000108,0.0,-0.023228,0.017305,0.017310,0.017305,...,0.0,0.004,0.05,0.523599,0.35,1.0,0.0,14.0,0.141853,0.0
2,0.06,295.188870,545.443402,6.040768,0.000483,0.0,-0.032652,0.007880,0.021380,0.007885,...,0.0,0.004,0.05,0.523599,0.35,1.0,0.0,14.0,0.147535,0.0
3,0.08,295.219241,545.464650,6.040430,0.000144,0.0,-0.023909,0.016624,0.016629,0.030122,...,0.0,0.004,0.05,0.523599,0.35,1.0,0.0,14.0,0.175993,0.0
4,0.10,294.775878,545.336549,6.046899,0.006614,0.0,-0.023999,0.016533,0.016539,0.016539,...,0.0,0.004,0.05,0.523599,0.35,1.0,0.0,14.0,0.235072,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,3.92,293.945746,544.807711,6.036504,-0.003781,0.0,-0.042591,0.011418,0.011412,0.038360,...,0.0,0.004,0.05,0.523599,0.35,1.0,0.0,14.0,0.123996,0.0
196,3.94,293.922169,544.797187,6.066272,0.025987,0.0,0.053093,0.039654,0.026166,0.053143,...,0.0,0.004,0.05,0.523599,0.35,1.0,0.0,14.0,0.115760,0.0
197,3.96,293.805378,544.748563,6.037079,-0.003206,0.0,-0.041767,0.012243,0.012242,0.025759,...,0.0,0.004,0.05,0.523599,0.35,1.0,0.0,14.0,0.196119,0.0
198,3.98,294.217929,544.887316,6.060329,0.020044,0.0,-0.066423,-0.012382,0.001107,0.028040,...,0.0,0.004,0.05,0.523599,0.35,1.0,0.0,14.0,0.330612,0.0
